In [ ]:
CASE = 0
BOW = 1
PATH = '/content/drive/My Drive/NLP/DataSet da Primeira Competição/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install unidecode

import nltk

nltk.download('stopwords')

stopwords_pt = nltk.corpus.stopwords.words('portuguese')
stopwords_en = nltk.corpus.stopwords.words('english')
stopwords_sp = nltk.corpus.stopwords.words('spanish')
stopwords = stopwords_pt + stopwords_en + stopwords_sp

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
import numpy as np

def get_data(path):
  df_train = pd.read_csv(path + 'train.csv')
  df_test = pd.read_csv(path+ 'test.csv')

  print('Loading Dataset...')
  print(df_train.shape)
  print(df_test.shape)

  X, y = (pd.concat([df_train.Text, df_test.Text])), pd.concat([df_train.Classificacao, pd.Series(np.zeros(df_test.shape[0],))])

  return X, y

In [ ]:
import re
import unidecode as uc

from collections import Counter
from nltk.tokenize.casual import casual_tokenize 
from nltk import RegexpTokenizer as RT

X,y = get_data(PATH)

print('Cleaning text...')
vocabulary = []
bow = []

for text in X:
  sentence = text

  tokens = casual_tokenize(sentence) # strip_handles=True remove @user
  tokens = [x for x in tokens if x not in stopwords]

  vocabulary += [" ".join([x for x in tokens])]
# for text in X:
#   tokens =  RT(r'\w+').tokenize(re.sub(r'http\S+|@\S+|rt\s|#|[0-9]+|_', '', uc.unidecode(text.lower()))) 
#   # tokens =  RT(r'\w+').tokenize(re.sub(r'http\S+|@\S+|rt\s|#', '', uc.unidecode(text.lower()))) 
  
#   tokens = [token for token in tokens if token not in stopwords_pt and token not in stopwords_en and token not in stopwords_sp]
#   tokens = casual_tokenize(' '.join(tokens), reduce_len=True)

#   vocabulary += [' '.join([x for x in tokens])]

print('OK')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

if BOW:
  print('Creating BoW')
  count_vect = CountVectorizer()
  data = count_vect.fit_transform(vocabulary)
# print("Our vocabulary: ", count_vect.vocabulary_)
else:
  print('Creating TFidf')
  vectorizer = TfidfVectorizer(min_df=1)
  data = vectorizer.fit_transform(vocabulary)

print(data.shape)
# 6053

Loading Dataset...
(6559, 11)
(1640, 10)
Cleaning text...
OK
Creating BoW
(8199, 12934)


In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(data, y, test_size=0.2, shuffle=False)

if CASE:  
  Xtrain, Xtest, ytrain, ytest = train_test_split(Xtrain, ytrain, test_size=0.2, shuffle=False)

print(Xtrain.shape, ytrain.shape)
print(Xtest.shape, ytest.shape)

(6559, 12934) (6559,)
(1640, 12934) (1640,)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

In [ ]:
print('Grid search SGDClassifier')
parameters = {'loss':('hinge', 'log', 'perceptron')}
model = SGDClassifier(verbose=0, random_state=52, eta0=1)
scores = ['precision']


for score in scores:
  clf = GridSearchCV(model, parameters, scoring='%s_macro' % score, cv=5, verbose=3)
  clf.fit(Xtrain, ytrain)

  print("Best parameters:")
  print(clf.best_params_)

  print("Grid scores: ")
  stds = clf.cv_results_['std_test_score']
  means = clf.cv_results_['mean_test_score']
  for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))


# #save results
# ypred = clf.predict(Xtest)

# final = pd.DataFrame(df_test.Id)
# final['Category'] = ypred

# final.to_csv('/content/drive/My Drive/Datasets/ufg-nlp-2021-competition-1/teste.csv',index=False)

Grid search SGDClassifier
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] loss=hinge ......................................................
[CV] .......................... loss=hinge, score=0.964, total=   0.1s
[CV] loss=hinge ......................................................
[CV] .......................... loss=hinge, score=0.965, total=   0.1s
[CV] loss=hinge ......................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] .......................... loss=hinge, score=0.965, total=   0.1s
[CV] loss=hinge ......................................................
[CV] .......................... loss=hinge, score=0.965, total=   0.1s
[CV] loss=hinge ......................................................
[CV] .......................... loss=hinge, score=0.960, total=   0.1s
[CV] loss=log ........................................................
[CV] ............................ loss=log, score=0.961, total=   0.1s
[CV] loss=log ........................................................
[CV] ............................ loss=log, score=0.968, total=   0.1s
[CV] loss=log ........................................................
[CV] ............................ loss=log, score=0.963, total=   0.1s
[CV] loss=log ........................................................
[CV] ............................ loss=log, score=0.964, total=   0.1s
[CV] loss=log ........................................................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    1.0s finished
